In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # TinyLlama's sweet spot (256-512 tokens)
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", " ", ""]
)

In [ ]:
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Persist locally (creates ./chroma_db folder)
vectorstore = Chroma(
    collection_name="study_notes",
    embedding_function=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),  # Uses Ollama's model
    persist_directory="./chroma_db"  # Critical for later reuse
)

In [10]:
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.65, "k": 4}  # Tune these!
)

In [11]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(
    model="tinyllama",
    temperature=0.3,  # Lower = more factual (critical for study aid)
    num_ctx=2048      # TinyLlama's max context window
)

In [12]:
from langchain.prompts import PromptTemplate

template = """You are a STUDY TUTOR analyzing COURSE MATERIALS. 
Use ONLY the context below to answer. If unsure, say "I don't know based on my materials".

Context:
{context}

Question: {question}

Answer concisely in 2-3 sentences:"""

QA_PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])

In [13]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_PROMPT},
    return_source_documents=True  # For citations!
)

In [14]:
result = qa_chain.invoke("Explain backpropagation in neural networks")
print("Answer:", result["result"])
print("\nSource Chunks:")
for doc in result["source_documents"]:
    print(f"- {doc.metadata['source']} (page {doc.metadata.get('page', 'N/A')})")

No relevant docs were retrieved using the relevance score threshold 0.65


Answer: Based on the given context, the question asks for an explanation of backpropagation in neural networks. The answer should be concise and specific to the question asked. In this case, it would be something like "backpropagation is a technique used in neural networks to update weights based on errors made by the network during training."

Source Chunks:
